In [1]:
import numpy as np
import pandas as pd
import math
import time
import re
import os
from skimage import util
from scipy.io import wavfile
from scipy import signal
from scipy import stats
import xarray as xr
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Create data frame of annotation info

In [3]:
annot_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/Analysis/Excel files/USV/annot_info_df.csv"
data = pd.read_csv(annot_path)
annot_info = pd.DataFrame(data = data)
print(annot_info.shape)
annot_info.head()

(552, 8)


,Unnamed: 0,Animal,Group,Session,Begin Time (s),Annotation,Begin Time (s)_1000,time_stamp
0,0,533,5,CPApair,376.574455,low slug,376574.45450,376560.0
1,1,533,5,CPApair,46.306579,low slug,46306.57941,46305.0
2,2,533,5,CPApair,243.272865,low slug,243272.86510,243270.0
3,3,533,5,CPApair,149.708324,low slug,149708.32400,149692.5
4,4,533,5,CPApair,243.176192,low slug,243176.19170,243157.5


Find path names for each netcdf file corresponding to wav file that has annotated data

In [4]:
netcdf_path = 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets'

In [5]:
path_names = []
files = os.listdir(netcdf_path)
for file in files: 
        path_names.append(netcdf_path + "/" + file)

path_names

['C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/533_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/534_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/535_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/542_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/543_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/554_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/555_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/559_xr_Dataset.nc',
 'C:/Users/Schindler/Documents/ProgrammingFun/USV_python/Datasets/600_xr_Dataset.nc']

In [ ]:
def get_annot_bool(path, annot_info):
    
    #get animal name from path
    name = re.search("\d\d\d", path).group(0)
    print(name)
    #open xarray Dataset, create 
    data = xr.open_dataset(path)
    slice_indexs = pd.DataFrame(data = data['slices'].values)

    annot_time_stamps = annot_info[annot_info['Animal'] ==  int(name)]['time_stamp'].sort_values()
    annot_bool = slice_indexs.isin(annot_time_stamps.values)

    return annot_bool

In [9]:
path = path_names[2]
name = re.search("\d\d\d", path).group(0)
print(name)
data = xr.open_dataset(path)
num_slices = data['slices'].shape[0]
print(num_slices)
slice_indexs = pd.DataFrame(data = data['slices'].values)
print(len(slice_indexs))

535
26666
26666


In [28]:
annot_time_stamps = annot_info[annot_info['Animal'] ==  int(name)]['time_stamp'].sort_values()

In [26]:
annot_bool = slice_indexs.isin(annot_time_stamps.values)
annot_bool_dic = {}
annot_bool_dic[name] = annot_bool

In [ ]:
spec_slice = data.sel(slices = 45.0)['__xarray_dataarray_variable__'].values
spec_slice.sum()

In [ ]:
x = data['__xarray_dataarray_variable__']
x[2]

In [ ]:
data.sel()

In [ ]:
spec_slice.mean(dim='times')
x

In [ ]:
spec_slice = data.sel(slices = 90.0)
spec_slice

In [ ]:
data.to_pandas()

In [ ]:
#select animal to process (need to process individually due to memory contraints)
path = path_names[0]
name = re.search("\d\d\d", path).group(0)

#process wav file of animal corresponding to annotations
print(str('Begin processing animal # ' + name))

#create slices
start = time.time()
samp_freq, sig_data, slices, steps = create_slice_from_wav(path, 6250, 0.9)
end = time.time()
print(str('Slices created in ' + str(end - start) + '  seconds'))

#create spectrograms
start = time.time()
spec_slices, freqs_spec, times = create_spec_from_slice_array(slices, spec_window=128, NFFT=512, samp_freq=250000, steps=5625)
end = time.time()
print(str('Spectrograms created in ' + str(end - start) + '  seconds'))

#create xarray Dataset
start = time.time()
slices_Dataset = create_xarray_dataset_from_dic(spec_slices, freqs_spec, times)
end = time.time()
print(str('xarray created in ' + str(end - start) + '  seconds'))

#save
start = time.time()
slices_Dataset.to_netcdf(name + '_xr_Dataset.nc')
end = time.time()
print(str('xarray saved in ' + str(end - start) + '  seconds'))

In [ ]:
slices_Dataset.sel(freq=slice(0,20000))[4,:,:].plot()